# Data Cleaning and Normalization

In this notebook, we perform all necessary cleaning tasks to prepare the ICU patient outcomes data for analysis, which will be conducted in other notebooks.

## 1. Import necessary libraries, have function definitions

In [179]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

In [180]:
# Clean and impute missing values in the specified columns
def clean_and_impute_columns(df_raw, selected_columns, threshold_ratio=0.2):
    df = df_raw.copy()
    df = df[df["cohort"] == "vap"]
    df = df[df["repeat"] == 1]
    df = df.dropna(subset=['balf_PD-L1_V1_imputed'])
    print(df.shape)
    selected_df_raw = df[selected_columns]
    
    # Identify columns with missing values
    missing_values = selected_df_raw.isnull().sum()
    missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
    #print(missing_values)
    
    # Drop columns with excessive missing values
    columns_with_many_missing_values = missing_values[missing_values >= threshold_ratio * len(df)].index
    selected_df = selected_df_raw.drop(columns=columns_with_many_missing_values)
    
    # Impute remaining missing values with column mean
    selected_df_imputed = selected_df.fillna(selected_df.median())
    
    return selected_df_imputed

In [181]:
# Identify and remove columns that are highly correlated with each other
def remove_highly_correlated_columns(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    printed_pairs = set()
    
    # Identify and print highly correlated pairs
    for col in corr_matrix.columns:
        for row in corr_matrix.index:
            if abs(corr_matrix.loc[row, col]) > threshold and row != col:
                pair = tuple(sorted([row, col]))
                if pair not in printed_pairs:
                    print(f"Highly correlated pair: {pair} ({corr_matrix.loc[row, col]})")
                    printed_pairs.add(pair)

    # Identify columns to drop
    upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

    # Drop highly correlated columns
    df_cleaned = df.drop(columns=to_drop)
    
    print(f"\nDropped {len(to_drop)}.")
    
    return df_cleaned

In [182]:
# Apply log transfation to a dataframe
def log2_transformation(df):
    log2_df = np.log2(df + 1)
    return log2_df

In [183]:
# Apply Z-score normalization to a dataframe
def z_score_normalization(df):
    scaler = StandardScaler()
    scaled_array = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaled_array, columns=df.columns)
    return scaled_df

## 2. Load the data

In [184]:
file_path = "../data/raw_data/Daily_merged_2025-02-27(in).csv"
sheet_name = "in"

# read the raw data
df_raw = pd.read_csv(file_path)
print(df_raw.shape)
df_raw.head()

(814, 2677)


,merged_id,subject_id,cohort,repeat,encoded_id,true_admit_date,admit_date_redcap,icu_admit_date_iths,icu_admit_source,icu_admit_type,...,pc_IL-10_proinf_V1_imputed,pc_IL-12p70_proinf_V1_imputed,pc_IL-13_proinf_V1_imputed,pc_IL-1?_proinf_V1_imputed,pc_IL-2_proinf_V1_imputed,pc_IL-4_proinf_V1_imputed,pc_IL-6_proinf_V1_imputed,pc_TNF-?_proinf_V1_imputed,pc_sRAGE_V1_imputed,pc_TNF-RI_V1_imputed
0,013f7716e4e32da88ac193198d5e063a30f3aa5e9ced9c...,hme_15,slicc,1,26aee58af5f8fecaa501ca4baa647553c84792f03e64a8...,2024-03-19T00:00:00.000000000,2024-03-19T18:12:00.000000000,2024-03-19T23:49:00.000000000,Emergency department,Trauma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,025ee0bffcaac7ae2e0971e7ba9b6f6caeceaba943fa46...,27,slicc,1,f14c884709a119a2ea317279f0b81199d942c77e08267c...,2022-02-07T00:00:00.000000000,2022-02-07T05:28:00.000000000,2022-02-07T04:11:00.000000000,Outside hospital transfer,Trauma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,044855276243ef8f79aef8effd3182fe23c893c1564ecc...,24,slicc,1,155085ab0c50e2225601297c0577f82e90fa8493c09155...,2021-12-02T00:00:00.000000000,2021-12-05T15:00:00.000000000,2021-12-05T15:00:00.000000000,Outside hospital transfer,Medical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,05825503bd131c8153ecbd44d7f44a5615bcac23cdcebc...,70,slicc,1,70c7c2f575353ab5329b3001a3202c0119e74da7f9be45...,2024-10-11T00:00:00.000000000,2024-10-11T07:44:00.000000000,2024-10-11T08:48:00.000000000,Emergency department,Medical,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,08c56505bbbc2327fc392392f2911b8a90ec5d2b239a0c...,8,slicc,1,40c9c999e80237b40f262d3fcc18dd3d313b0e3b896c00...,2021-01-20T00:00:00.000000000,2021-01-20T21:40:00.000000000,2021-01-20T18:57:00.000000000,Outside hospital transfer,Trauma,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814 entries, 0 to 813
Columns: 2677 entries, merged_id to pc_TNF-RI_V1_imputed
dtypes: float64(2260), int64(56), object(361)
memory usage: 16.6+ MB


In [186]:
df_raw.describe()

,repeat,intubated_at_outside_hospi,slicc1_sliccy-vap2_vap3_hme4,hsv_cmv_sample_analyzed_1yes,HME_0no_1yes,cytof_bal_b1,cytof_bal_b2,cytof_pbmc_b1,cytof_pbmc_b2,smoking,...,pc_IL-10_proinf_V1_imputed,pc_IL-12p70_proinf_V1_imputed,pc_IL-13_proinf_V1_imputed,pc_IL-1?_proinf_V1_imputed,pc_IL-2_proinf_V1_imputed,pc_IL-4_proinf_V1_imputed,pc_IL-6_proinf_V1_imputed,pc_TNF-?_proinf_V1_imputed,pc_sRAGE_V1_imputed,pc_TNF-RI_V1_imputed
count,814.000000,45.000000,155.000000,155.000000,646.000000,49.000000,49.000000,49.00000,49.000000,502.000000,...,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,1.084767,1.022222,1.903226,0.845161,0.063467,0.408163,0.061224,0.77551,0.510204,0.874502,...,2.683260,0.497297,5.776273,0.590696,0.610753,0.091409,42.055691,3.350314,1571.400784,32760.857877
std,0.304034,0.260148,0.858678,0.362923,0.243991,0.496587,0.242226,0.42157,0.505076,1.793253,...,3.832488,0.412624,1.478580,0.866729,0.637236,0.052682,78.224271,2.559785,645.420125,38649.710278
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.274940,0.022435,3.270744,0.041778,0.062188,0.016906,3.820443,0.973806,567.341122,5380.410762
25%,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000,...,0.782265,0.223716,4.615804,0.214974,0.188938,0.058103,9.511728,1.676127,1078.904659,9763.675809
50%,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,1.00000,1.000000,0.000000,...,1.170845,0.431738,5.728350,0.306524,0.464540,0.073738,14.952700,2.126856,1399.799342,14561.330565
75%,1.000000,1.000000,3.000000,1.000000,0.000000,1.000000,0.000000,1.00000,1.000000,1.000000,...,2.498728,0.620684,6.767929,0.552650,0.780558,0.113170,31.846341,4.363101,1915.402182,33371.459293
max,3.000000,2.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,9.000000,...,16.958756,1.994190,8.312335,3.842504,3.137333,0.263269,320.948966,12.402581,2821.506157,131531.422600


There are missing data in the dataset. Since we don’t need all the data for each of our analysis, we will select specific columns and handle the missing values separately.

## 3. Biomarker Analysis

Manually I made a list of columns that falls under the category of biomarkers!

We will now clean the dataset specifically for dataset analysis.

In [187]:
biomarker_columns = list(col for col in df_raw.columns if col.startswith('balf_'))
imputed_biomarker_columns = list(col for col in biomarker_columns if col.endswith('V1_imputed'))
filtered_imputed_biomarker_columns = list(col for col in imputed_biomarker_columns if not col.startswith('balf_IL-8'))
filtered_imputed_biomarker_columns = list(col for col in imputed_biomarker_columns if not col.startswith('balf_IL-10'))
keywords = [
    "Amphiregulin", "Calprotectin", "CD163", "G-CSF", "GM-CSF",
    "IL-12/IL-23p40", "IL-15", "IL-16", "IL-17A", "IL-1α", "IL-1β",
    "IL-6", "IL-7", "IL-8_chemo", "IL-8_proinflam", "IL-10", "IP-10",
    "MCP-1", "MCP-4", "MIP-1α", "MIP-1β", "PD-L1", "sRAGE",
    "TARC", "TNF-RI", "TNF-α", "VEGF"
]

filtered_imputed_biomarker_columns = [col for col in filtered_imputed_biomarker_columns if any(keyword in col for keyword in keywords)]
print(filtered_imputed_biomarker_columns)
print(len(filtered_imputed_biomarker_columns))

['balf_Amphiregulin_V1_imputed', 'balf_Calprotectin_V1_imputed', 'balf_CD163_V1_imputed', 'balf_IL-8_chemo_V1_imputed', 'balf_IP-10_chemo_V1_imputed', 'balf_MCP-1_chemo_V1_imputed', 'balf_MCP-4_chemo_V1_imputed', 'balf_TARC_chemo_V1_imputed', 'balf_GM-CSF_V1_imputed', 'balf_IL-12/IL-23p40_V1_imputed', 'balf_IL-15_V1_imputed', 'balf_IL-16_V1_imputed', 'balf_IL-17A_V1_imputed', 'balf_IL-7_V1_imputed', 'balf_VEGF_V1_imputed', 'balf_G-CSF_V1_imputed', 'balf_PD-L1_V1_imputed', 'balf_IL-6_proinf_V1_imputed', 'balf_sRAGE_V1_imputed', 'balf_TNF-RI_V1_imputed']
20


In [188]:
df_biomarker_cleaned = clean_and_impute_columns(df_raw, filtered_imputed_biomarker_columns)
df_biomarker_cleaned.head()

(466, 2677)


,balf_Amphiregulin_V1_imputed,balf_Calprotectin_V1_imputed,balf_CD163_V1_imputed,balf_IL-8_chemo_V1_imputed,balf_IP-10_chemo_V1_imputed,balf_MCP-1_chemo_V1_imputed,balf_MCP-4_chemo_V1_imputed,balf_TARC_chemo_V1_imputed,balf_GM-CSF_V1_imputed,balf_IL-12/IL-23p40_V1_imputed,balf_IL-15_V1_imputed,balf_IL-16_V1_imputed,balf_IL-17A_V1_imputed,balf_IL-7_V1_imputed,balf_VEGF_V1_imputed,balf_G-CSF_V1_imputed,balf_PD-L1_V1_imputed,balf_IL-6_proinf_V1_imputed,balf_sRAGE_V1_imputed,balf_TNF-RI_V1_imputed
78,4.443273,9.716818e+04,23508.10788,290.841257,59.007167,231.834988,1.277551,3.518362,0.733159,0.428807,2.270383,63.556487,0.026801,1.759322,254.991609,23.117045,4.467335,0.753105,994.126338,164.696351
79,113.614985,2.820000e+07,117855.10570,1.994190,0.141996,1.994190,0.022226,0.046973,1.713464,34.656009,2.978609,1583.271188,58.973952,2.246199,524.154669,1808.750514,49.939963,561.831149,1901.941090,2719.605322
80,80.998467,1.280000e+07,24079.04721,40626.142370,176.915152,281.815611,2.855676,2.652712,0.412834,4.596715,0.289479,209.854546,2.447253,0.569876,274.684635,395.028450,8.403314,558.609463,3133.153270,2119.757305
81,3.155196,4.072710e+05,20304.46508,1274.876365,47.676634,912.818422,8.883987,2.182495,0.652714,1.994190,2.786929,26.113594,0.449550,0.206973,66.815244,72.087988,5.649633,4.997066,575.709100,265.647159
82,149.195013,5.550000e+07,12140.80450,24344.398590,931.252352,92.837395,4.013650,5.451239,0.365938,6.783007,0.532258,1612.890040,3.567370,2.504409,978.848901,590.741358,11.898387,128.664177,90.290345,2227.872938


For the analysis, I want to evaluate whether removing highly correlated values impacts the model's performance and leads to any improvements.

So I will conduct one set of analyses using the dataset with correlated fields and another set using the dataset without them.

In [189]:
df_biomarker_cleaned_notcorrelated = remove_highly_correlated_columns(df_biomarker_cleaned, 0.5)
df_biomarker_cleaned_notcorrelated.head()

Highly correlated pair: ('balf_Amphiregulin_V1_imputed', 'balf_PD-L1_V1_imputed') (0.5348968106016682)
Highly correlated pair: ('balf_Calprotectin_V1_imputed', 'balf_IL-8_chemo_V1_imputed') (0.758032612969414)
Highly correlated pair: ('balf_Calprotectin_V1_imputed', 'balf_IL-16_V1_imputed') (0.5145889776225913)
Highly correlated pair: ('balf_Calprotectin_V1_imputed', 'balf_VEGF_V1_imputed') (0.8199788264197414)
Highly correlated pair: ('balf_Calprotectin_V1_imputed', 'balf_TNF-RI_V1_imputed') (0.7467899034767365)
Highly correlated pair: ('balf_CD163_V1_imputed', 'balf_PD-L1_V1_imputed') (0.7748718208072802)
Highly correlated pair: ('balf_IL-8_chemo_V1_imputed', 'balf_VEGF_V1_imputed') (0.6560829925199557)
Highly correlated pair: ('balf_IL-8_chemo_V1_imputed', 'balf_TNF-RI_V1_imputed') (0.6025803994523974)
Highly correlated pair: ('balf_IL-12/IL-23p40_V1_imputed', 'balf_IP-10_chemo_V1_imputed') (0.571135154201538)
Highly correlated pair: ('balf_IP-10_chemo_V1_imputed', 'balf_PD-L1_V1_im

,balf_Amphiregulin_V1_imputed,balf_Calprotectin_V1_imputed,balf_CD163_V1_imputed,balf_IP-10_chemo_V1_imputed,balf_MCP-1_chemo_V1_imputed,balf_MCP-4_chemo_V1_imputed,balf_TARC_chemo_V1_imputed,balf_IL-15_V1_imputed,balf_IL-17A_V1_imputed,balf_IL-7_V1_imputed
78,4.443273,9.716818e+04,23508.10788,59.007167,231.834988,1.277551,3.518362,2.270383,0.026801,1.759322
79,113.614985,2.820000e+07,117855.10570,0.141996,1.994190,0.022226,0.046973,2.978609,58.973952,2.246199
80,80.998467,1.280000e+07,24079.04721,176.915152,281.815611,2.855676,2.652712,0.289479,2.447253,0.569876
81,3.155196,4.072710e+05,20304.46508,47.676634,912.818422,8.883987,2.182495,2.786929,0.449550,0.206973
82,149.195013,5.550000e+07,12140.80450,931.252352,92.837395,4.013650,5.451239,0.532258,3.567370,2.504409


Some models perform better when features are on a similar scale. 

Let us perform log2 transformation and Z-score normalization to ensure that the features have a mean of 0 and a standard deviation of 1 for both the datasets.

In [190]:
log2_df_biomarker_cleaned = log2_transformation(df_biomarker_cleaned)
scaled_df_biomarker_cleaned = z_score_normalization(log2_df_biomarker_cleaned)
scaled_df_biomarker_cleaned.head()

,balf_Amphiregulin_V1_imputed,balf_Calprotectin_V1_imputed,balf_CD163_V1_imputed,balf_IL-8_chemo_V1_imputed,balf_IP-10_chemo_V1_imputed,balf_MCP-1_chemo_V1_imputed,balf_MCP-4_chemo_V1_imputed,balf_TARC_chemo_V1_imputed,balf_GM-CSF_V1_imputed,balf_IL-12/IL-23p40_V1_imputed,balf_IL-15_V1_imputed,balf_IL-16_V1_imputed,balf_IL-17A_V1_imputed,balf_IL-7_V1_imputed,balf_VEGF_V1_imputed,balf_G-CSF_V1_imputed,balf_PD-L1_V1_imputed,balf_IL-6_proinf_V1_imputed,balf_sRAGE_V1_imputed,balf_TNF-RI_V1_imputed
0,-1.470130,-1.498284,-0.246658,-1.129279,-0.614070,-0.030338,-0.866099,-0.440824,-0.655745,-1.162979,0.375604,-0.695447,-1.099571,0.286584,0.352247,-1.095234,-0.887325,-1.598728,-0.288758,-1.412866
1,0.803770,0.911863,1.228127,-3.101678,-2.734490,-2.711906,-1.388472,-1.679053,-0.042735,1.728370,0.789723,1.435168,2.252914,0.530009,0.845505,1.136749,1.368696,1.138960,0.049796,0.753702
2,0.553875,0.576147,-0.224706,0.996641,-0.032357,0.089440,-0.522827,-0.620921,-0.935185,0.064120,-1.590510,0.092549,-0.101333,-0.558272,0.403123,0.351343,-0.339183,1.136237,0.310366,0.560863
3,-1.671626,-0.889214,-0.380679,-0.493922,-0.726075,0.811831,0.090998,-0.737614,-0.720736,-0.498054,0.685410,-1.272986,-0.815376,-0.952069,-0.559623,-0.522124,-0.689435,-1.015348,-0.573650,-1.044524
4,1.005517,1.199630,-0.851120,0.776083,0.854145,-0.590079,-0.351586,-0.139260,-0.981345,0.360495,-1.226081,1.447499,0.130569,0.644662,1.273656,0.558921,-0.019721,0.442616,-1.536270,0.599358


In [191]:
# dataset with highly correlated columns removed
scaled_df_biomarker_cleaned_notcorrelated = z_score_normalization(df_biomarker_cleaned_notcorrelated)
scaled_df_biomarker_cleaned_notcorrelated.head()

,balf_Amphiregulin_V1_imputed,balf_Calprotectin_V1_imputed,balf_CD163_V1_imputed,balf_IP-10_chemo_V1_imputed,balf_MCP-1_chemo_V1_imputed,balf_MCP-4_chemo_V1_imputed,balf_TARC_chemo_V1_imputed,balf_IL-15_V1_imputed,balf_IL-17A_V1_imputed,balf_IL-7_V1_imputed
0,-0.492915,-0.679906,-0.364026,-0.398849,-0.526441,-0.291255,-0.222259,0.109407,-0.320342,-0.083610
1,0.109179,0.444489,0.703246,-0.426586,-0.820222,-0.300766,-0.271759,0.528131,1.692955,0.079621
2,-0.070704,-0.171665,-0.357568,-0.343292,-0.462556,-0.279299,-0.234603,-1.061765,-0.237674,-0.482387
3,-0.500018,-0.667498,-0.400267,-0.404188,0.343985,-0.233626,-0.241308,0.414804,-0.305904,-0.604054
4,0.305407,1.536762,-0.492616,0.012145,-0.704107,-0.270526,-0.194698,-0.918226,-0.199417,0.166189


## 4. Save data in .csv

In [192]:
# Save the cleaned and normalized biomarker data
scaled_df_biomarker_cleaned.to_csv("../data/clean_data/scaled_biomarker_data.csv", index=False)

In [193]:
# Save the cleaned and normalized biomarker data (with highly correlated columns removed)
scaled_df_biomarker_cleaned_notcorrelated.to_csv("../data/clean_data/scaled_biomarker_data_notcorrelated.csv", index=False)